In [1]:
import os
os.chdir('../..')

In [2]:
import convokit
import pickle
import json

In [3]:
DATA_DIR = "data_fixed"

In [4]:
os.chdir('convokit/tensors')

In [5]:
with open(os.path.join(DATA_DIR, 'rank_to_factors.p'), 'rb') as f:
    rank_to_factors = pickle.load(f)

In [6]:
with open(os.path.join(DATA_DIR, 'hg_features.p'), 'rb') as f:
    hg_features = pickle.load(f)

with open(os.path.join(DATA_DIR, 'subreddits.p'), 'rb') as f:
    subreddits = pickle.load(f)

In [7]:
from sklearn.preprocessing import StandardScaler
import numpy as np
scaler = StandardScaler()

In [8]:
def get_anomalous_points(factor_full, idx):
    factor = factor_full[:, idx]
    reshaped = factor.reshape((factor.shape[0], 1))
    scaled = scaler.fit_transform(reshaped)
    pos_pts = np.argwhere(scaled.reshape(factor.shape[0]) > 1.5).flatten()
    neg_pts = np.argwhere(scaled.reshape(factor.shape[0]) < -1.5).flatten()
    return pos_pts, neg_pts

In [9]:
time_factor = rank_to_factors[9][0] # (9, 9)
thread_factor = rank_to_factors[9][1] # (10000, 9)
feature_factor = rank_to_factors[9][2] # (140, 9)

In [10]:
with open(os.path.join(DATA_DIR, 'convo_ids.p'), 'rb') as f:
    thread_ids = pickle.load(f)

In [11]:
from convokit import Corpus, download
corpus = Corpus(filename="longreddit_construction/long-reddit-corpus")

In [12]:
for convo in corpus.iter_conversations():
    for idx, utt in enumerate(convo.get_chronological_utterance_list()):
        utt.meta['order'] = idx

In [13]:
def get_convo_details(convo):
    print("Subreddit: {}".format(convo.get_utterance(convo.id).meta['subreddit']))
    convo.print_conversation_structure(lambda utt: str(utt.meta['order']) + ". " + utt.user.id, limit=20)

In [14]:
convos = list(corpus.iter_conversations())

In [15]:
from convokit import HyperConvo

In [16]:
import random

## Inspecting threads from Factor 1

In [17]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [18]:
rank = time_factor.shape[1]
num_examples = 3

In [19]:
for idx in range(rank):
    print("#########################################################")
    print(color.BOLD + "Inspecting threads from factor {}".format(idx+1) + color.END)
    print()
    pos_threads, neg_threads = get_anomalous_points(thread_factor, idx)
    
    print(color.BOLD + color.GREEN + "Positive examples" + color.END)
    print()
    try:
        for thread_idx in random.sample(list(pos_threads), num_examples):
            get_convo_details(convos[thread_idx])
            print()
    except ValueError:
        pass
    print()
    print(color.BOLD + color.RED + "Negative examples" + color.END)
    
    try:
        for thread_idx in random.sample(list(neg_threads), num_examples):
            get_convo_details(convos[thread_idx])
            print()
    except ValueError:
        pass
    print("#########################################################")
    print()

#########################################################
Inspecting threads from factor 1

Positive examples

Subreddit: baseball
1. staalsarebrothers
    2. pitbaseball
        3. brown_boot
            4. pitbaseball
                5. brown_boot
                    6. pitbaseball
                        7. brown_boot
                            8. pitbaseball
                                9. brown_boot
                                    10. pitbaseball
                                        11. brown_boot
                                            12. pitbaseball
        13. Armani_Chode
            14. Crazed_Chemist
            18. pitbaseball
        15. Corzare
            16. jfoster15
                17. Corzare
            19. pitbaseball
                20. joshm509

Subreddit: pokemontrades
1. toxiclaws
    2. imthorrbo2
        3. toxiclaws
            4. imthorrbo2
                5. toxiclaws
                    6. imthorrbo2
                        7. toxiclaws
  